<a href="https://colab.research.google.com/github/Omsai-2000/Python-Projects/blob/main/Roster_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#-------------------------------------------------------------------------------------------------#
# Author  : Omsai Yatthapu                                                                        #
# Purpose : Python script for generating multiple Excel files from single Excel file              #
#-------------------------------------------------------------------------------------------------#

#--------------------------Importing Required Libraries/Modules-----------------------------------#

import pandas as pd
from datetime import datetime

# If we use any other application/Environment for executing this script, use below lines
# And remove Upload files statement from colab
#file_path = r'C:\Users\oyattha\Desktop\Roster dev\Roster_Aug2023.xls'
#df = pd.read_excel(file_path)

# This below code only useful for uploading files into google colab for reading excel.

from google.colab import files

print("\033[1mPlease Upload Employee Roster: \033[0m")
upload_first_file = files.upload()
file_name1 = list(upload_first_file.keys())[0]
Main_excel = pd.read_excel(file_name1)


#-----------------------Main script starts from Here-----------------------------------------------#

#---Below code skip the first 2 rows and slice the first 2 characters over the dataFrame---------#

all_row_sliced_data = []

for index, row in Main_excel.iterrows():
    if index < 2:
        continue

    row_sliced_data = []
    first_column_value = row[Main_excel.columns[0]]
    row_sliced_data.append(first_column_value)

    for column_name, cell_value in row.items():
        if isinstance(cell_value, str) and column_name != Main_excel.columns[0]:
            sliced_data = cell_value[0:2]
            row_sliced_data.append(sliced_data)
    all_row_sliced_data.append(row_sliced_data)
df1 = pd.DataFrame(all_row_sliced_data)

#-------------------------------------------------------------------------------------------------#

#---------------Below function is used to map the specific string values in DataFrame-------------#

def map_shift_code(value):
    if value == 'G1':
        return 'General Shift 1'
    elif value == 'G2':
        return 'General Shift 2'
    elif value == 'D\n':
        return 'Shift D'
    elif value == 'WO':
        return 'Weekly Off'
    else:
        return value

for i in df1[:]:
  df1[i] = df1[i].apply(map_shift_code)
df2_sorted=df1.sort_values(by=[df1.columns[0]], key=lambda col: col.str[0])
df2=df2_sorted.T
df2=df2.iloc[1:]

#--------------------------------------------------------------------------------------------------#

#-----------------Below code changes the Date formats from Main Excel------------------------------#

from datetime import datetime
columns=Main_excel.columns[1:]
data = []
month_mapping = {
    "Jan": "J",
    "Feb": "F",
    "Mar": "M",
    "Apr": "A",
    "May": "M",
    "Jun": "J",
    "Jul": "J",
    "Aug": "A",
    "Sep": "S",
    "Oct": "O",
    "Nov": "N",
    "Dec": "D"
}
for date_str in columns:
    input_date = datetime.strptime(date_str, "%d %b %y %a")
    parts = date_str.split()
    abbreviated_month = parts[1]
    month_prefix = month_mapping.get(abbreviated_month, "Unknown")
    output_date_str = month_prefix + input_date.strftime("%Y-%m-%d")
    data.append(output_date_str)
df3 = pd.DataFrame(data)

#-------------------------------------------------------------------------------------------------#

#------------------Below code Matches the index and concatenate both Dataframes-------------------#

df3.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

result = pd.concat([df3, df2], axis=1)
result=result.T

result.columns = result.iloc[0]
result = result[1:]

#--------------------------------------------------------------------------------------------------#

#---Below code import the Employee Details and sort the Emp Names and concat with Main dataFrame---#

print("\033[1mPlease Upload Employee details sheet: \033[0m")
upload_second_file = files.upload()
file_name2 = list(upload_second_file.keys())[0]
emp_df = pd.read_excel(file_name2)
emp_df_sorted = emp_df.sort_values(by=[emp_df.columns[1]], key=lambda col: col.str.split().str[2])
emp_df = emp_df_sorted.reset_index(drop=True)

# Use below 3 lines of code if we use other than Google colab
#emp_df = pd.read_excel('Emp_details.xlsx')
#emp_df = emp_df.sort_values(by=[emp_df.columns[1]], key=lambda col: col.str.split().str[2])
#emp_df = emp_df.reset_index(drop=True)

emp_df.reset_index(drop=True, inplace=True)
result.reset_index(drop=True, inplace=True)
combined_df = pd.concat([emp_df, result], axis=1)

#--------------------------------------------------------------------------------------------------#

#--------Below code take individual combinations with Column---------------------------------------#

combinations_by_row = []
for index, row in combined_df.iterrows():
    row_combinations = []
    for col in combined_df.columns:
      row_combinations.append((col, row[col]))
    combinations_by_row.append(row_combinations)

#--------------------------------------------------------------------------------------------------#

#-----------Below code delete the entire column if their is Holiday or Leave present in value------#

all_modified_combinations = []
for i in range(len(combinations_by_row)):
    row_combinations = combinations_by_row[i]
    df4 = pd.DataFrame(row_combinations, columns=['Column', 'Value'])
    df4 = df4[~df4['Value'].str.contains('H|L', na=False)]
    modified_combinations = [(col, val) for col, val in zip(df4['Column'], df4['Value'])]
    all_modified_combinations.append(modified_combinations)

employee_names = combined_df['EmployeeName'].tolist()
Emp_last_name = [name.split()[2] if len(name.split()) >= 3 else '' for name in employee_names]
transposed_dataframes = [pd.DataFrame(data).T for data in all_modified_combinations]

#--------------------------------------------------------------------------------------------------#

#---------------------Below code will generate Output files ---------------------------------------#

for Emp_last_name, Separte_excel in zip(Emp_last_name, transposed_dataframes):
    file_name = f'RosteringData4-{Emp_last_name}.xlsx'
    Separte_excel.to_excel(file_name, index=False,header=False)

#------------------------------Script Ends Here----------------------------------------------------#

Please Upload Employee Roster: 


Saving Roster_Sep2023.xls to Roster_Sep2023 (2).xls
Please Upload Employee details sheet: 


Saving Emp_details.xlsx to Emp_details (22).xlsx
